# Cancer Detection using VGG-16 
----

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms

## Load in the datasets
----
Info on how to obtain the datasets is contained in the README.md. Data augmentation techniques are applied in the data loader here to be applied to the training data. 

In [2]:
### Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
train_transfoms = transforms.Compose([transforms.Resize((224 , 224))
                              ,transforms.RandomRotation(degrees = 10)
                              ,transforms.RandomHorizontalFlip(p=0.2)
                              ,transforms.RandomGrayscale(p=0.2)
                              ,transforms.RandomVerticalFlip(p=0.2)
                              ,transforms.ToTensor()
                              ,transforms.Normalize(mean=[0.485, 0.456, 0.406]
                                                   , std=[0.229, 0.224, 0.225])
                              ])

test_val_transfomrs = transforms.Compose([transforms.Resize((224 , 224))
                              ,transforms.ToTensor()
                              ,transforms.Normalize(mean=[0.485, 0.456, 0.406]
                                                   , std = [0.229, 0.224, 0.225])
                              ])

batch_size = 4 #About all my little GPU can handle 
num_workers = 0

train_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\train'
                                  , transform=train_transfoms)
test_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\test'
                                 , transform=test_val_transfomrs)
valid_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\valid'
                                  , transform=test_val_transfomrs)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

loaders = dict(train=train_loader, test=test_loader, valid=valid_loader)

#### There are three class of skin lesion to look at in the problem.
    - melanoma
    - nevus
    - seborrheic_keratosis

In [3]:
!ls C:\\Users\\diarm\\Downloads\\cancer_detection\\data\\train

melanoma
nevus
seborrheic_keratosis


In [4]:
resnet50 = models.resnet50(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    resnet50 = resnet50.cuda()

#### The final classifier can be viewed here. 
For transfer learning we will change this final layer to suit our case and retrain it with our data. 

# Transfer learning
----
The VGG-16 network was retrieved from pytorch.models - [you can find it here](https://pytorch.org/docs/stable/torchvision/models.html#id2)

A new fully connected layer is appended to the VGG-16 structure. We are replacing the current fully connected layer which takes in 25088 features and has 1000 outputs. 
The VGG16 model has been trained on the ImageNet dataset which is a very large collection of images curated into 1 of 1000 classes.

I'm going to use the underlying layers of the network to piggyback on the extraction of features from an image, and just train the final layer to classify the images from our dataset.
Retraining all of the parameters in the network was avoided due to lack of personal computational power :(

In [5]:
### Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
train_transfoms = transforms.Compose([transforms.Resize((224 , 224))
                              ,transforms.RandomRotation(degrees = 10)
                              ,transforms.RandomHorizontalFlip(p=0.2)
                              ,transforms.RandomGrayscale(p=0.2)
                              ,transforms.RandomVerticalFlip(p=0.2)
                              ,transforms.ToTensor()
                              ,transforms.Normalize(mean=[0.485, 0.456, 0.406]
                                                   , std=[0.229, 0.224, 0.225])
                              ])

test_val_transfomrs = transforms.Compose([transforms.Resize((224 , 224))
                              ,transforms.ToTensor()
                              ,transforms.Normalize(mean=[0.485, 0.456, 0.406]
                                                   , std = [0.229, 0.224, 0.225])
                              ])

batch_size = 32 
#num_workers = 0

train_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\train'
                                  , transform=train_transfoms)
test_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\test'
                                 , transform=test_val_transfomrs)
valid_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\valid'
                                  , transform=test_val_transfomrs)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

loaders = dict(train=train_loader, test=test_loader, valid=valid_loader)

#### There are three class of skin lesion to look at in the problem.
    - melanoma
    - nevus
    - seborrheic_keratosis

In [6]:
!ls C:\\Users\\diarm\\Downloads\\cancer_detection\\data\\train

melanoma
nevus
seborrheic_keratosis


In [7]:
vgg16 = models.vgg16(pretrained=True)

# training on CPU as too many parameters for my smal GPU
use_cuda = False

# move model to GPU if CUDA is available
if use_cuda:
    vgg16 = vgg16.cuda()

In [8]:
vgg16.state_dict

<bound method Module.state_dict of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size

In [20]:
for param in vgg16.parameters():
    param.requires_grad = False

# Two final fully connected layer for output
classifier = nn.Sequential(nn.Linear(25088, 2048)
                           ,nn.ReLU()
                           ,nn.Dropout(0.2)
                           ,nn.Linear(2048, 1024)
                           ,nn.ReLU()
                           ,nn.Dropout(0.2)
                           ,nn.Linear(1024, 3)
                          )
vgg16.classifier = classifier
# Transfer to GPU if available
if use_cuda:
    vgg16 = vgg16.to('cuda')

In [21]:
params_to_update = []
for name,param in vgg16.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(name)    

In [22]:
params_to_update

['classifier.0.weight',
 'classifier.0.bias',
 'classifier.3.weight',
 'classifier.3.bias',
 'classifier.6.weight',
 'classifier.6.bias']

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001)

In [24]:
# A helper function to show classification accuracy
def label_accuracy(model, validation_loader, epoch, use_cuda):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, targets in validation_loader:
            if use_cuda:
                images, targets = images.cuda(), targets.cuda()
            outputs = model(images)
            _, preds = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (preds == targets).sum().item()
    print('Accuray test at epoch {}\tLabel Accuracy : - {} %'.format(epoch, round(correct / total, 2)))

# Training 
----

In [25]:
# Defining the trainig function
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # clear the accumulated gradients 
            optimizer.zero_grad()
            
            # inception net produces two outputs, aux output handles vanishing gradient
            outputs = model(data)
            #loss calculation https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
            loss = criterion(outputs, target)
            
            # gradient of the loss with respect to the parameters
            loss.backward()
            # perform the parameter update (update the weights)
            optimizer.step()
            
            ## record the average training loss
            # train loss - adds current loss to accumulated loss (averaged over batch size)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # get prediction
            output = model(data)
            # calculate the loss
            loss = criterion(output, target)
            ## update the average validation loss
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
            torch.save(model.state_dict(), save_path) #add save path here
            valid_loss_min = valid_loss
            # get epoch accuracy 
            #label_accuracy(model, loaders['valid'], epoch, use_cuda)
        
        # learning rate decay scheduled here if necessary
        #scheduler.step()
        
    return model

In [15]:
# let 'er riiiip
trained_model = train(10, loaders, vgg16, optimizer, criterion, use_cuda, save_path='./vgg16_temp_models/model_cancer_detection.pt')

Epoch: 1 	Training Loss: 1.079473 	Validation Loss: 0.852840

Validation loss decreased (inf --> 0.852840).  Saving model ...

Epoch: 2 	Training Loss: 0.675858 	Validation Loss: 0.783597

Validation loss decreased (0.852840 --> 0.783597).  Saving model ...

Epoch: 3 	Training Loss: 0.623265 	Validation Loss: 0.881511

Epoch: 4 	Training Loss: 0.588937 	Validation Loss: 0.760433

Validation loss decreased (0.783597 --> 0.760433).  Saving model ...

Epoch: 5 	Training Loss: 0.557401 	Validation Loss: 0.698879

Validation loss decreased (0.760433 --> 0.698879).  Saving model ...

Epoch: 6 	Training Loss: 0.547175 	Validation Loss: 0.986227

Epoch: 7 	Training Loss: 0.538457 	Validation Loss: 0.902956

Epoch: 8 	Training Loss: 0.495600 	Validation Loss: 0.939307

Epoch: 9 	Training Loss: 0.477355 	Validation Loss: 1.031939

Epoch: 10 	Training Loss: 0.447920 	Validation Loss: 0.968628
